## PSO Algorithm Design

This notebook was used to design PSO algorithm and test it with dummy values

In [3]:
import math
import numpy as np

In [9]:

class Particle(object):
    '''
    Particle class  
    
    Parameters:
    lb: lower bound - list with size of hyperparameter numbers each element contain a lower limit
    ub: upper \bound - list with size of hyperparameter numbers each element contain a upper limit
    types: list of types of each hyperparameter (if int or float)
    '''
    def __init__(self, lb, ub, types) -> None:
        
        assert len(lb) == len(ub),'different ranges for lb and ub'
        assert len(lb) == len(types), 'types and bounds have diff range'
        self.lb = np.array(lb)
        self.ub = np.array(ub)
        self.types = types
        self.inttypes = [] #which hyperparam indexes are integer?
        self.V = []      
        self.pos = []
        self.fitness_values = []
        
        #initialize first position
        pos_1 = []
        v_1 = []
        for i in range(len(self.lb)):
            assert self.lb[i] <= self.ub[i]
            
            if self.lb[i] == self.ub[i]:
                pos_1.append(self.lb[i])
                v_1.append(0)    
                continue
            rng = self.ub[i] - self.lb[i]
            #position initialized randomly with uniform distr.
            rand_pos = np.random.uniform(self.lb[i], self.ub[i])
            
            #velocity initialized with normal distribution and range of ub-lb as 3 standard deviations
            rand_v = np.random.normal(0, rng/3)
            if types[i] == int:
                rand_pos = int(rand_pos)
                rand_v = int(rand_v)
                self.inttypes.append(i)
            pos_1.append(rand_pos)
            v_1.append(rand_v)
                

        self.pos.append(np.array(pos_1))
        self.V.append(np.array(v_1))      
        self.pbest = []
        self.pbest_score = []

    def update_pbest(self, fitness_val):
        '''
        Compares fitness value with current pbest_Score if smaller updates pbest
        
        Parameters:
        fitness_val: loss score from the model training
        '''        
        self.fitness_values.append(fitness_val)
        
        if not self.pbest_score or fitness_val < self.pbest_score[-1]:
            self.pbest_score.append(fitness_val)
            self.pbest.append(self.pos[-1])
        else: 
            self.pbest_score.append(self.pbest_score[-1])
            self.pbest.append(self.pbest[-1])
        
    def update_V(self, w, c1, c2, gbest):
        '''
        Update velocity according with classic PSO algorithm
                
        Parameters:
        w: inercia
        c1: acceleration for cognitive component
        c2: acceleration for social component
        gbest: global best position
        '''  
        r1 = np.random.uniform(size=np.array(self.pos[-1]).shape)
        r2 = np.random.uniform(size=np.array(self.pos[-1]).shape)
        print(r2) 
        print(r1)
        print(self.pbest[-1] - self.pos[-1])
        print(gbest - self.pos[-1])
        self.V.append(w*self.V[-1] + c1*r1*(self.pbest[-1] - self.pos[-1]) 
        + c2*r2*(gbest - self.pos[-1]))
        
    def update_pos(self):
        '''
        Updates position according with classic PSO algorithm
                
        '''  
        self.pos.append(self.pos[-1] + self.V[-1])
        
        for idx in self.inttypes:
            self.pos[-1][idx] = int(self.pos[-1][idx])
            
        # if position values gets larger or smaller than limits clip the value
        clip_lower = self.pos[-1] < self.lb
        clip_upper = self.pos[-1] > self.ub
        
        self.pos[-1][clip_lower] = self.lb[clip_lower]            
        self.pos[-1][clip_upper] = self.lb[clip_upper]
                     
      
            




class PSO(object):
    '''
    Update velocity according with classic PSO algorithm
            
    Parameters:
    w: inercia
    hyperparam: dictionary with keys as names and value as lists lowest and highest\n 
    value for each hyperparameters
    c1: acceleration for cognitive component
    c2: acceleration for social component
    num particles: integer number 
    '''     
    def __init__(self, w: float, hyperparam: dict,  c1=0.5, c2=0.5, num_particles=20 ):
        #initial paramters
        self.w = w  #inertia weight
        self.c1 = c1 #acceleration 1
        self.c2 = c2 #acceleration 2
        #particles info
        self.hyperparam = hyperparam
        #get type int or float of each hyperparam to use later 
        self.hyperparam_types = []
        for param_space in self.hyperparam.values():
            if  type(param_space[1]) == int:
                self.hyperparam_types.append(int)
            else:
                self.hyperparam_types.append(float)
                            
        self.num_particles = num_particles
        self.particles = []
        self.gbest = []
        self.gbest_score = []
        #defining lower and upper bounds for each hyperparam
        self.lb = [hyperparam[i][0] for i in hyperparam]
        self.ub = [hyperparam[i][-1] for i in hyperparam]
        
    def initialize_particles(self):


        for i in range(self.num_particles):
            self.particles.append(Particle(self.lb, self.ub, self.hyperparam_types))

    def update_gbest(self, g, position, fitness_val):
        '''
        Update global best if fitness_val < gbest_score looking at current\n and future generations
        
        Parameters:
        
        g - generation
        position - current particle position
        fitness_val - value of current particle_position 
        
        '''    
        if fitness_val < self.gbest_score[g]:
            self.gbest_score[g] = fitness_val
            self.gbest[g] = position
            for i in range(g+1,len(self.gbest)): 
                self.gbest_score[i] = self.gbest_score[g]
                self.gbest[i] = self.gbest[g]

    def run(self, train_model, generations: int):
        '''
        Run the complete PSO generations finding best hyperparameter combination
            
        Parameters:

        train_model - function to train model and return loss_val        
        generations - integer number of 

        
        '''
        self.gbest = [0 for g in range(generations)]
        self.gbest_score = [math.inf for g in range(generations)]
        
        for g in range(generations):
            #fitness function 
            
            
            for particle in self.particles:
                #calculate loss error for each particle
                fitness_value = train_model(particle.pos[-1])
                
                #update local and global best 
                particle.update_pbest(fitness_value)
                self.update_gbest(g, particle.pos[-1], fitness_value)               
                
                #update V
                particle.update_V(self.w, self.c1, self.c2, self.gbest[-1])
                particle.update_pos()
        
        
     

### dummy values test

In [10]:

lower_bound, upper_bound = np.array([1,2,0.18,4]), np.array([14,15,16,17])
types = [int, int, float, int]

In [11]:
lower_bound

array([1.  , 2.  , 0.18, 4.  ])

In [12]:
pso = PSO(w=0.5, hyperparam= {'dropout':[0.1, 0.9], 'lr':[1e-5, 1e-3], 'image_size': [100, 160]}, c1=0.5, c2=0.5, num_particles=5 )

In [13]:
pso.initialize_particles()

In [254]:
pso.particles

In [272]:
pso.gbest = [0 for g in range(3)]
pso.gbest_score = [math.inf for g in range(3)]

In [273]:
pso.gbest

[0, 0, 0]

In [274]:
fv = 0.999
for g in range(3):
    
    for p in pso.particles:
        fv = fv - 0.1*np.random.random()
        p.update_pbest(fv)
        pso.update_gbest(g, p.pos[-1], fv)
        p.update_V(0.5, 1.1, 1.1, pso.gbest[-1])
        p.update_pos()

[0.75524978 0.5090871  0.97248172]
[0.31935729 0.58677436 0.17068822]
[0. 0. 0.]
[0. 0. 0.]
[array([8.54274987e-01, 2.68815508e-04, 1.12000000e+02])]
v  [array([ 1.5289885e-01, -3.9721238e-04, -1.0000000e+01]), array([ 7.64494252e-02, -1.98606190e-04, -5.00000000e+00])]
[array([8.54274987e-01, 2.68815508e-04, 1.12000000e+02]), array([9.30724412e-01, 7.02093179e-05, 1.07000000e+02])]
[0.0605243  0.14397636 0.90456134]
[0.23993522 0.59878868 0.77062362]
[0. 0. 0.]
[0. 0. 0.]
[array([7.40020916e-01, 8.52007175e-04, 1.35000000e+02])]
v  [array([ 2.98919304e-01,  1.74470697e-04, -3.50000000e+01]), array([ 1.49459652e-01,  8.72353484e-05, -1.75000000e+01])]
[array([7.40020916e-01, 8.52007175e-04, 1.35000000e+02]), array([8.89480568e-01, 9.39242523e-04, 1.17500000e+02])]
[0.70615634 0.13966564 0.77873873]
[0.92787272 0.38719943 0.38639559]
[0. 0. 0.]
[0. 0. 0.]
[array([2.80270616e-01, 8.10573291e-04, 1.22000000e+02])]
v  [array([-3.55882436e-01, -2.73221638e-04,  0.00000000e+00]), array([-1.7

In [275]:
pso.particles[1].pbest

[array([7.40020916e-01, 8.52007175e-04, 1.35000000e+02]),
 array([8.89480568e-01, 9.39242523e-04, 1.17000000e+02]),
 array([1.00000000e-01, 9.82860197e-04, 1.08000000e+02])]

In [276]:
pso.particles[3].pbest

[array([2.55121100e-01, 9.68480171e-04, 1.22000000e+02]),
 array([4.54281352e-01, 1.00000000e-05, 1.19000000e+02]),
 array([5.53861477e-01, 6.62187146e-05, 1.17000000e+02])]

In [277]:
pso.gbest

[array([1.26331923e-01, 2.77524149e-04, 1.11000000e+02]),
 array([1.00000000e-01, 4.81341518e-04, 1.19000000e+02]),
 array([1.00000000e-01, 5.83250202e-04, 1.23000000e+02])]

In [263]:
pso.gbest_score

[0.721766040110506, 0.5679765362999213, 0.2485090411479492]

In [148]:
np.random.random()

0.4442079005160786